In [3]:
from paho.mqtt import client as mqtt_client
from datetime import datetime
import csv
import os
import json
from adtk.detector import SeasonalAD
from adtk.data import validate_series
from adtk.visualization import plot
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from adtk.detector import LevelShiftAD
warnings.filterwarnings("ignore", message="Setting an item of incompatible dtype")

In [4]:
def on_message_data_append(id,row):

    csv_file = f"data_csv_{id}.csv"

    if not os.path.exists(csv_file):
        
        with open(csv_file, mode='w', newline='') as file:
            file.write('date,x,y,z\n')

        df = pd.read_csv(csv_file)
        row_count = len(df)

        if(row_count >= 120):
            df.drop(0)

        new_row = {'date': row[0], 'x': row[1], 'y': row[2], 'z': row[3]}

        df = df.append(new_row, ignore_index=True)

        df.to_csv(csv_file, index=False)

In [11]:
def getAnomalysLevelShift(id):
    csv_file = f"normal_data_accelerometer.csv"
    
    sleeping_data = pd.read_csv(csv_file)

    # Convert 'date' column to datetime
    sleeping_data['date'] = pd.to_datetime(sleeping_data['date'])

    # Set 'date' column as the index
    sleeping_data.set_index('date', inplace=True)

    # get each axis data
    x_axis_data = sleeping_data["x"]
    y_axis_data = sleeping_data["y"]
    z_axis_data = sleeping_data["z"]
    
    # validating series
    x_train = validate_series(x_axis_data)
    y_train = validate_series(y_axis_data)
    z_train = validate_series(z_axis_data)

    # Create and fit the LevelShiftAD detector for x_train
    level_shift_ad_x = LevelShiftAD(c=6.0, side='both', window=3)
    anomalies_x = level_shift_ad_x.fit_detect(x_train)
    
    # Create and fit the LevelShiftAD detector for y_train
    level_shift_ad_y = LevelShiftAD(c=6.0, side='both', window=3)
    anomalies_y = level_shift_ad_y.fit_detect(y_train)
    
    # Create and fit the LevelShiftAD detector for z_train
    level_shift_ad_z = LevelShiftAD(c=6.0, side='both', window=3)
    anomalies_z = level_shift_ad_z.fit_detect(z_train)
    
    # Filter out NaN values from anomalies
    non_nan_anomalies_x = anomalies_x.dropna()
    non_nan_anomalies_y = anomalies_y.dropna()
    non_nan_anomalies_z = anomalies_z.dropna()
    
    anomalies_x_dates = []
    anomalies_y_dates = []
    anomalies_z_dates = []
    
    # Append the dates of the anomalies of the x-axis
    for anomaly_idx, anomaly in non_nan_anomalies_x.items():
        if anomaly:
            anomalies_x_dates.append(anomaly_idx)
    
    # Append the dates of the anomalies of the y-axis
    for anomaly_idx, anomaly in non_nan_anomalies_y.items():
        if anomaly:
            anomalies_y_dates.append(anomaly_idx)
    
    # Append the dates of the anomalies of the z-axis
    for anomaly_idx, anomaly in non_nan_anomalies_z.items():
        if anomaly:
            anomalies_z_dates.append(anomaly_idx)
    
     # Example usage:
    print("Anomalies in X-axis:", anomalies_x_dates)
    print("Anomalies in Y-axis:", anomalies_y_dates)
    print("Anomalies in Z-axis:", anomalies_z_dates)
    


In [12]:
print(getAnomalysLevelShift("123"))

Anomalies in X-axis: [Timestamp('2024-03-26 15:50:28.125963'), Timestamp('2024-03-26 15:50:29.253715'), Timestamp('2024-03-26 15:50:30.198182')]
Anomalies in Y-axis: [Timestamp('2024-03-26 15:50:28.125963'), Timestamp('2024-03-26 15:50:29.253715'), Timestamp('2024-03-26 15:50:30.198182'), Timestamp('2024-03-26 15:50:31.112705')]
Anomalies in Z-axis: []
None
